# Tutorial 7: MEV and Front-Running

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/blockchain-tokens-and-incentives/blob/main/notebooks/tutorial_07_mev_and_front_running.ipynb)

---

> *"The exchange is trustless. But the queue to reach it is not. He who orders the line extracts from those who stand in it."*
>
> — Brenn Auster, *On Ordering and Extraction* (Year 873)

---

## The Observer Effect

Tutorial 06 built the AMM — an automated exchange that replaces inspectors with arithmetic. No trust required. No bribery possible. The formula holds.

But Auster discovered a new vulnerability, one he hadn't anticipated: **the mempool**.

When a guild member submits a trade to the AMM, the transaction doesn't execute immediately. It enters a waiting area — the **mempool** — where it sits until a validator includes it in the next block. And the mempool is public. Every validator can see every pending transaction before it executes.

This creates an asymmetry: validators (and anyone with mempool access) can see what traders intend to do *before* it happens. And they can insert their own transactions *first*.

**Miner Extractable Value (MEV)** — now called **Maximum Extractable Value** — is the profit that can be extracted by reordering, inserting, or censoring transactions within a block. It's the "observer effect" of transparent blockchains: the act of observing pending transactions changes their outcome.

Auster called this "the tax of transparency" — the price the guild pays for its trustless exchange being readable by everyone, including those who order the trades.

## Learning Objectives

In this tutorial, you will:

1. **Understand MEV** — what it is, why it exists, and who profits
2. **Simulate front-running** — see how observing pending trades enables extraction
3. **Build a sandwich attack** — the classic MEV strategy in detail
4. **Analyze the MEV dataset** — 150 extraction events across 6 types
5. **Implement detection strategies** — identify MEV in transaction logs
6. **Model Flashbots-style ordering** — fair ordering as a mitigation

**Technical Concepts:**
- Miner/Maximum Extractable Value (MEV)
- Front-running, back-running, and sandwich attacks
- Arbitrage and liquidation as MEV
- Transaction ordering and block construction
- Flashbots and fair sequencing

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

mev_log = pd.read_csv(BASE_URL + "mev_extraction_log.csv")
amm_pools = pd.read_csv(BASE_URL + "amm_liquidity_pools.csv")

print(f"MEV extraction log: {len(mev_log)} events")
print(f"MEV types: {list(mev_log['mev_type'].unique())}")
print(f"Extractors: {mev_log['extractor'].nunique()} unique")
print(f"Pools affected: {list(mev_log['pool'].unique())}")

## The MEV Taxonomy

Not all MEV is equal. Some forms are harmful (sandwich attacks steal from traders), some are neutral (arbitrage corrects prices), and some are arguably beneficial (liquidations prevent protocol insolvency).

| Type | Description | Victim? | Harm Level |
|------|-------------|---------|------------|
| **Sandwich attack** | Front-run + back-run a victim's swap | Yes | High |
| **Front-run** | Trade before victim to move price | Yes | High |
| **Back-run** | Trade after victim to capture residual | No direct victim | Low |
| **Liquidation** | Liquidate undercollateralized positions | Indirect | Medium |
| **Arbitrage** | Correct cross-pool price differences | None | Beneficial |
| **JIT liquidity** | Add/remove liquidity around large swaps | None | Debated |

In [ ]:
# MEV type distribution
type_stats = mev_log.groupby('mev_type').agg(
    count=('event_id', 'count'),
    total_profit=('extractor_profit', 'sum'),
    avg_profit=('extractor_profit', 'mean'),
    total_victim_loss=('victim_loss', 'sum'),
    avg_gas=('gas_cost', 'mean'),
    success_rate=('successful', 'mean'),
    avg_net=('net_profit', 'mean')
).sort_values('total_profit', ascending=False)

print("MEV Extraction by Type:")
print(f"{'Type':25s} {'Count':>5s} {'Total Profit':>13s} {'Victim Loss':>12s} "
      f"{'Avg Net':>9s} {'Success':>8s}")
for mev_type, row in type_stats.iterrows():
    print(f"{mev_type:25s} {int(row['count']):5d} {row['total_profit']:13.1f} "
          f"{row['total_victim_loss']:12.1f} {row['avg_net']:9.1f} {row['success_rate']:7.0%}")

print(f"\nTotal MEV extracted: {mev_log['extractor_profit'].sum():,.1f}")
print(f"Total victim losses: {mev_log['victim_loss'].sum():,.1f}")
print(f"Total gas spent: {mev_log['gas_cost'].sum():,.1f}")
print(f"Net profit (after gas): {mev_log['net_profit'].sum():,.1f}")

In [ ]:
# Visualize MEV landscape
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# By type: count and profit
colors = {'sandwich_attack': 'firebrick', 'front_run': 'darkorange',
          'back_run': 'goldenrod', 'liquidation': 'steelblue',
          'arbitrage': 'forestgreen', 'just_in_time_liquidity': 'mediumpurple'}

type_order = type_stats.index.tolist()
bar_colors = [colors.get(t, 'gray') for t in type_order]

axes[0, 0].barh(type_order, type_stats['count'], color=bar_colors, alpha=0.8, edgecolor='black')
axes[0, 0].set_xlabel('Number of Events')
axes[0, 0].set_title('MEV Events by Type')

axes[0, 1].barh(type_order, type_stats['total_profit'], color=bar_colors, alpha=0.8, edgecolor='black')
axes[0, 1].set_xlabel('Total Extracted Profit')
axes[0, 1].set_title('Total Profit by MEV Type')

# Profit vs gas cost scatter
for mev_type in mev_log['mev_type'].unique():
    subset = mev_log[mev_log['mev_type'] == mev_type]
    axes[1, 0].scatter(subset['gas_cost'], subset['extractor_profit'],
                        c=colors.get(mev_type, 'gray'), label=mev_type,
                        alpha=0.6, edgecolors='black', s=40)
axes[1, 0].set_xlabel('Gas Cost')
axes[1, 0].set_ylabel('Extractor Profit')
axes[1, 0].set_title('Profit vs Gas Cost')
axes[1, 0].legend(fontsize=7, loc='upper left')
axes[1, 0].grid(alpha=0.3)

# Net profit distribution
axes[1, 1].hist(mev_log['net_profit'], bins=30, color='steelblue', alpha=0.8, edgecolor='black')
axes[1, 1].axvline(x=0, color='firebrick', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Net Profit (after gas)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Net Profit Distribution')
axes[1, 1].grid(alpha=0.3)

plt.suptitle('MEV Extraction Landscape', fontsize=13)
plt.tight_layout()
plt.show()

unprofitable = (mev_log['net_profit'] <= 0).sum()
print(f"Unprofitable extractions: {unprofitable}/{len(mev_log)} ({unprofitable/len(mev_log):.0%})")
print(f"MEV is not free money — gas costs can exceed profits.")

## Simulating a Front-Run

A **front-run** is the simplest MEV attack: see a pending trade, execute the same trade first, then profit from the price impact.

1. Victim submits: "Buy 50 iron_ore for salt_crystal"
2. Attacker sees this in the mempool
3. Attacker buys iron_ore first (moves price up)
4. Victim's trade executes at a worse price
5. Attacker profits from the price increase

In [ ]:
class SimpleAMM:
    """Minimal AMM for MEV demonstration."""
    FEE = 0.003
    
    def __init__(self, reserve_a, reserve_b):
        self.reserve_a = reserve_a
        self.reserve_b = reserve_b
    
    @property
    def k(self):
        return self.reserve_a * self.reserve_b
    
    @property
    def price(self):
        return self.reserve_b / self.reserve_a
    
    def swap_b_for_a(self, amount_b):
        """Deposit B, receive A. Returns amount of A."""
        fee = amount_b * self.FEE
        net_b = amount_b - fee
        k = self.k
        new_b = self.reserve_b + net_b
        new_a = k / new_b
        amount_a = self.reserve_a - new_a
        self.reserve_a = new_a
        self.reserve_b = new_b + fee
        return amount_a
    
    def swap_a_for_b(self, amount_a):
        """Deposit A, receive B. Returns amount of B."""
        fee = amount_a * self.FEE
        net_a = amount_a - fee
        k = self.k
        new_a = self.reserve_a + net_a
        new_b = k / new_a
        amount_b = self.reserve_b - new_b
        self.reserve_a = new_a + fee
        self.reserve_b = new_b
        return amount_b

# === Front-Running Demo ===
print("=== FRONT-RUNNING SIMULATION ===")
print()

# WITHOUT front-running (fair execution)
fair_pool = SimpleAMM(1000, 1600)
print(f"Initial price: {fair_pool.price:.4f} salt per iron")

victim_amount = 80  # Victim wants to buy iron with 80 salt
victim_received_fair = fair_pool.swap_b_for_a(victim_amount)
print(f"\nFAIR EXECUTION:")
print(f"  Victim deposits: {victim_amount} salt")
print(f"  Victim receives: {victim_received_fair:.4f} iron")
print(f"  Execution price: {victim_amount / victim_received_fair:.4f} salt per iron")

# WITH front-running
attacked_pool = SimpleAMM(1000, 1600)
print(f"\nFRONT-RUN EXECUTION:")

# Step 1: Attacker front-runs with 50 salt
attacker_buy = 50
attacker_iron = attacked_pool.swap_b_for_a(attacker_buy)
print(f"  1. Attacker buys: {attacker_buy} salt → {attacker_iron:.4f} iron")
print(f"     Price after: {attacked_pool.price:.4f} salt per iron")

# Step 2: Victim trades at worse price
victim_received_attacked = attacked_pool.swap_b_for_a(victim_amount)
print(f"  2. Victim buys: {victim_amount} salt → {victim_received_attacked:.4f} iron")
print(f"     Price after: {attacked_pool.price:.4f} salt per iron")

# Step 3: Attacker sells iron back
attacker_salt_back = attacked_pool.swap_a_for_b(attacker_iron)
print(f"  3. Attacker sells: {attacker_iron:.4f} iron → {attacker_salt_back:.4f} salt")

attacker_profit = attacker_salt_back - attacker_buy
victim_loss = victim_received_fair - victim_received_attacked

print(f"\nRESULT:")
print(f"  Attacker profit: {attacker_profit:+.4f} salt")
print(f"  Victim received: {victim_received_attacked:.4f} iron (vs {victim_received_fair:.4f} fair)")
print(f"  Victim loss: {victim_loss:.4f} iron ({victim_loss/victim_received_fair:.1%} worse)")

## The Sandwich Attack

A **sandwich attack** is a front-run paired with a back-run — the attacker wraps ("sandwiches") the victim's transaction:

1. **Front-run:** Buy before the victim → push price up
2. **Victim trade:** Executes at inflated price
3. **Back-run:** Sell after the victim → lock in profit

The attacker profits from the price movement they caused. The victim pays more than they should have.

In [ ]:
def simulate_sandwich(reserve_a, reserve_b, victim_amount, attacker_amount, fee=0.003):
    """Simulate a sandwich attack and return detailed results."""
    # Fair execution (no attack)
    fair = SimpleAMM(reserve_a, reserve_b)
    fair_received = fair.swap_b_for_a(victim_amount)
    fair_price = victim_amount / fair_received
    
    # Sandwich attack
    pool = SimpleAMM(reserve_a, reserve_b)
    price_before = pool.price
    
    # 1. Front-run
    front_iron = pool.swap_b_for_a(attacker_amount)
    price_after_front = pool.price
    
    # 2. Victim trade
    victim_iron = pool.swap_b_for_a(victim_amount)
    price_after_victim = pool.price
    
    # 3. Back-run
    back_salt = pool.swap_a_for_b(front_iron)
    price_after_back = pool.price
    
    return {
        'fair_received': fair_received,
        'fair_price': fair_price,
        'victim_received': victim_iron,
        'victim_price': victim_amount / victim_iron,
        'victim_loss_iron': fair_received - victim_iron,
        'victim_loss_pct': (fair_received - victim_iron) / fair_received * 100,
        'attacker_profit': back_salt - attacker_amount,
        'attacker_cost': attacker_amount,
        'price_trajectory': [price_before, price_after_front, price_after_victim, price_after_back],
    }

# Run sandwich with varying attacker sizes
victim_trade = 80
attacker_sizes = [10, 30, 50, 80, 120, 200]

print(f"Sandwich Attack Analysis (victim trade: {victim_trade} salt)\n")
print(f"{'Attacker Size':>14s} {'Profit':>8s} {'Victim Loss':>12s} {'Victim Loss%':>13s}")

results = []
for size in attacker_sizes:
    r = simulate_sandwich(1000, 1600, victim_trade, size)
    results.append(r)
    print(f"{size:>12d} {r['attacker_profit']:>8.3f} "
          f"{r['victim_loss_iron']:>12.4f} {r['victim_loss_pct']:>12.2f}%")

In [ ]:
# Visualize sandwich attack mechanics
# Use the 80-unit attacker case
r = simulate_sandwich(1000, 1600, 80, 80)
prices = r['price_trajectory']
stages = ['Before', 'After\nFront-Run', 'After\nVictim', 'After\nBack-Run']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Price trajectory
bar_colors = ['steelblue', 'firebrick', 'darkorange', 'forestgreen']
ax1.bar(stages, prices, color=bar_colors, alpha=0.8, edgecolor='black')
ax1.set_ylabel('Price (salt per iron)')
ax1.set_title('Price During Sandwich Attack')
ax1.grid(alpha=0.3, axis='y')
for i, p in enumerate(prices):
    ax1.annotate(f'{p:.3f}', (i, p), ha='center', va='bottom', fontsize=9)

# Attacker profit vs size
profits = [r['attacker_profit'] for r in results]
victim_losses = [r['victim_loss_pct'] for r in results]

ax2_twin = ax2.twinx()
ax2.bar(range(len(attacker_sizes)), profits, color='firebrick', alpha=0.7,
        edgecolor='black', width=0.4, label='Attacker profit')
ax2_twin.plot(range(len(attacker_sizes)), victim_losses, 'bo-', linewidth=2,
              label='Victim loss %')

ax2.set_xticks(range(len(attacker_sizes)))
ax2.set_xticklabels(attacker_sizes)
ax2.set_xlabel('Attacker Front-Run Size')
ax2.set_ylabel('Attacker Profit (salt)', color='firebrick')
ax2_twin.set_ylabel('Victim Loss %', color='blue')
ax2.set_title('Sandwich Attack: Profit vs Front-Run Size')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Larger front-runs move the price more, increasing victim harm.")
print("But attacker profit plateaus — diminishing returns from fees and slippage.")

## Analyzing the MEV Dataset

Now let's dive into the pre-generated `mev_extraction_log.csv` — 150 real extraction events.

In [ ]:
# Who are the extractors?
extractor_stats = mev_log.groupby('extractor').agg(
    events=('event_id', 'count'),
    total_profit=('extractor_profit', 'sum'),
    total_gas=('gas_cost', 'sum'),
    net_profit=('net_profit', 'sum'),
    success_rate=('successful', 'mean'),
    favorite_type=('mev_type', lambda x: x.mode().iloc[0] if len(x) > 0 else 'unknown')
).sort_values('net_profit', ascending=False)

print("Top MEV Extractors:")
print(f"{'Extractor':20s} {'Events':>7s} {'Gross':>8s} {'Gas':>8s} {'Net':>8s} "
      f"{'Win%':>5s} {'Specialty':>20s}")
for name, row in extractor_stats.iterrows():
    print(f"{name:20s} {int(row['events']):7d} {row['total_profit']:8.1f} "
          f"{row['total_gas']:8.1f} {row['net_profit']:8.1f} "
          f"{row['success_rate']:4.0%} {row['favorite_type']:>20s}")

In [ ]:
# Who are the victims?
victims = mev_log[mev_log['victim'].notna() & (mev_log['victim_loss'] > 0)].copy()

victim_stats = victims.groupby('victim').agg(
    times_victimized=('event_id', 'count'),
    total_loss=('victim_loss', 'sum'),
    avg_loss=('victim_loss', 'mean'),
    worst_attack=('mev_type', lambda x: x.mode().iloc[0] if len(x) > 0 else 'unknown')
).sort_values('total_loss', ascending=False)

print(f"Most Victimized Traders:")
print(f"{'Victim':20s} {'Times':>6s} {'Total Loss':>11s} {'Avg Loss':>9s} {'Main Attack':>20s}")
for name, row in victim_stats.head(10).iterrows():
    print(f"{name:20s} {int(row['times_victimized']):6d} {row['total_loss']:11.1f} "
          f"{row['avg_loss']:9.1f} {row['worst_attack']:>20s}")

print(f"\nTotal victims: {victims['victim'].nunique()}")
print(f"Total victim losses: {victims['victim_loss'].sum():,.1f}")
print(f"Average loss per event: {victims['victim_loss'].mean():.1f}")

In [ ]:
# MEV by pool — which pools attract the most extraction?
pool_mev = mev_log.groupby('pool').agg(
    events=('event_id', 'count'),
    total_extracted=('extractor_profit', 'sum'),
    total_victim_loss=('victim_loss', 'sum'),
    avg_trade_value=('victim_trade_value', 'mean'),
    sandwich_count=('mev_type', lambda x: (x == 'sandwich_attack').sum()),
    arb_count=('mev_type', lambda x: (x == 'arbitrage').sum())
).sort_values('total_extracted', ascending=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

pool_names = [p.replace('/', '\n') for p in pool_mev.index]
ax1.barh(pool_names, pool_mev['total_extracted'], color='firebrick', alpha=0.8, edgecolor='black')
ax1.set_xlabel('Total MEV Extracted')
ax1.set_title('MEV Extraction by Pool')

# Harmful vs benign MEV by pool
harmful = mev_log[mev_log['mev_type'].isin(['sandwich_attack', 'front_run'])]
benign = mev_log[mev_log['mev_type'].isin(['arbitrage', 'back_run', 'just_in_time_liquidity'])]

harmful_by_pool = harmful.groupby('pool')['extractor_profit'].sum()
benign_by_pool = benign.groupby('pool')['extractor_profit'].sum()

pools = pool_mev.index.tolist()
pool_labels = [p.replace('/', '\n') for p in pools]
h_vals = [harmful_by_pool.get(p, 0) for p in pools]
b_vals = [benign_by_pool.get(p, 0) for p in pools]

x_pos = range(len(pools))
ax2.bar(x_pos, h_vals, color='firebrick', alpha=0.8, label='Harmful (sandwich/front-run)')
ax2.bar(x_pos, b_vals, bottom=h_vals, color='forestgreen', alpha=0.8, label='Benign (arb/back-run/JIT)')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(pool_labels, fontsize=8)
ax2.set_ylabel('Total Extracted')
ax2.set_title('Harmful vs Benign MEV')
ax2.legend(fontsize=8)
ax2.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## MEV Detection

Detecting MEV in transaction logs is critical for understanding system health. Let's build detection heuristics for each MEV type.

In [ ]:
def detect_sandwich(tx_log):
    """Detect potential sandwich attacks in a transaction log.
    Pattern: A buys, B buys (same direction), A sells — all in same block."""
    suspects = []
    
    blocks = tx_log.groupby('block_number')
    for block_num, block_txs in blocks:
        if len(block_txs) < 3:
            continue
        
        # Look for sandwich pattern in the block
        sandwich_events = block_txs[block_txs['mev_type'] == 'sandwich_attack']
        if len(sandwich_events) > 0:
            for _, event in sandwich_events.iterrows():
                suspects.append({
                    'block': block_num,
                    'extractor': event['extractor'],
                    'victim': event['victim'],
                    'profit': event['extractor_profit'],
                    'victim_loss': event['victim_loss'],
                    'front_run_size': event['front_run_amount'],
                    'back_run_size': event['back_run_amount'],
                    'confidence': 'high' if event['front_run_amount'] > 0 and event['back_run_amount'] > 0 else 'medium'
                })
    
    return pd.DataFrame(suspects)

def detect_front_runs(tx_log):
    """Detect front-running: extractor trades before victim in same direction."""
    suspects = []
    front_runs = tx_log[tx_log['mev_type'] == 'front_run']
    
    for _, event in front_runs.iterrows():
        profit_ratio = event['extractor_profit'] / max(event['victim_trade_value'], 1)
        suspects.append({
            'block': event['block_number'],
            'extractor': event['extractor'],
            'victim': event['victim'],
            'profit': event['extractor_profit'],
            'victim_trade': event['victim_trade_value'],
            'extraction_rate': profit_ratio,
            'confidence': 'high' if profit_ratio > 0.01 else 'low'
        })
    
    return pd.DataFrame(suspects)

# Run detectors
sandwich_suspects = detect_sandwich(mev_log)
frontrun_suspects = detect_front_runs(mev_log)

print(f"Detection Results:")
print(f"  Sandwich attacks detected: {len(sandwich_suspects)}")
print(f"  Front-runs detected: {len(frontrun_suspects)}")

if len(sandwich_suspects) > 0:
    print(f"\nTop 5 sandwich attacks by profit:")
    top = sandwich_suspects.sort_values('profit', ascending=False).head(5)
    for _, row in top.iterrows():
        print(f"  Block {int(row['block'])}: {row['extractor']} → {row['victim']} "
              f"(profit: {row['profit']:.1f}, victim loss: {row['victim_loss']:.1f})")

In [ ]:
# Detection difficulty analysis
print("Detection Difficulty by MEV Type:")
difficulty = mev_log.groupby(['mev_type', 'detection_difficulty']).size().unstack(fill_value=0)
print(difficulty)

print(f"\nOverall difficulty distribution:")
diff_counts = mev_log['detection_difficulty'].value_counts()
for diff, count in diff_counts.items():
    print(f"  {diff:12s} {count:4d} ({count/len(mev_log):.0%})")

# Profit by detection difficulty
diff_profit = mev_log.groupby('detection_difficulty')['net_profit'].agg(['mean', 'sum', 'count'])
print(f"\nProfitability by detection difficulty:")
for diff, row in diff_profit.iterrows():
    print(f"  {diff:12s} avg_net={row['mean']:7.1f}, total={row['sum']:8.1f}, n={int(row['count'])}")

print(f"\nHarder-to-detect MEV tends to be more sophisticated and profitable.")

## The Economics of MEV

MEV is not just a technical problem — it's an economic one. It represents a tax on every user of the AMM, paid to those who can observe and reorder transactions.

In [ ]:
# MEV as percentage of trading volume
total_trade_volume = mev_log['victim_trade_value'].sum()
total_mev = mev_log['extractor_profit'].sum()
total_victim_loss = mev_log['victim_loss'].sum()

mev_tax_rate = total_mev / total_trade_volume * 100
victim_tax_rate = total_victim_loss / total_trade_volume * 100

print(f"MEV Economics:")
print(f"  Total trade volume: {total_trade_volume:,.0f}")
print(f"  Total MEV extracted: {total_mev:,.0f} ({mev_tax_rate:.2f}% of volume)")
print(f"  Total victim losses: {total_victim_loss:,.0f} ({victim_tax_rate:.2f}% of volume)")
print(f"  Gas overhead: {mev_log['gas_cost'].sum():,.0f}")
print(f"  Net extraction: {mev_log['net_profit'].sum():,.0f}")

# Compare to AMM fees
amm_swaps = amm_pools[amm_pools['action'] == 'swap']
total_amm_fees = amm_swaps['fee'].sum()
print(f"\nFor comparison:")
print(f"  AMM fees collected (0.3% per swap): {total_amm_fees:,.0f}")
print(f"  MEV as % of AMM fees: {total_mev / max(total_amm_fees, 1) * 100:.0f}%")
print(f"\nMEV is a significant additional cost beyond swap fees.")

In [ ]:
# MEV over time (by block number)
mev_log_sorted = mev_log.sort_values('block_number')
mev_log_sorted['cumulative_profit'] = mev_log_sorted['extractor_profit'].cumsum()
mev_log_sorted['cumulative_loss'] = mev_log_sorted['victim_loss'].cumsum()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Cumulative extraction
ax1.plot(mev_log_sorted['block_number'], mev_log_sorted['cumulative_profit'],
         'firebrick', linewidth=2, label='Cumulative extraction')
ax1.plot(mev_log_sorted['block_number'], mev_log_sorted['cumulative_loss'],
         'darkorange', linewidth=2, label='Cumulative victim loss')
ax1.set_xlabel('Block Number')
ax1.set_ylabel('Cumulative Value')
ax1.set_title('Cumulative MEV Over Time')
ax1.legend()
ax1.grid(alpha=0.3)

# Individual events
for mev_type in mev_log['mev_type'].unique():
    subset = mev_log_sorted[mev_log_sorted['mev_type'] == mev_type]
    color = {'sandwich_attack': 'firebrick', 'front_run': 'darkorange',
             'back_run': 'goldenrod', 'liquidation': 'steelblue',
             'arbitrage': 'forestgreen', 'just_in_time_liquidity': 'mediumpurple'
            }.get(mev_type, 'gray')
    ax2.scatter(subset['block_number'], subset['net_profit'], c=color,
                label=mev_type, alpha=0.6, s=30, edgecolors='black')

ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax2.set_xlabel('Block Number')
ax2.set_ylabel('Net Profit')
ax2.set_title('Individual MEV Events Over Time')
ax2.legend(fontsize=7, loc='upper left')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Flashbots and Fair Ordering

> *"If the problem is that validators see the queue, the solution is to encrypt the queue. If the problem is that validators order the queue, the solution is to remove their discretion."*
>
> — Brenn Auster

**Flashbots** proposed a solution: instead of a public mempool, create a private channel where users submit transaction bundles to block builders. The builder includes bundles atomically (all or nothing), and users can specify fair ordering constraints.

In Auster's framework, this is **mechanism design applied to transaction ordering** — designing the block construction game so that harmful extraction is impossible or unprofitable.

In [ ]:
class FairOrderingSimulator:
    """Compare different transaction ordering policies."""
    
    def __init__(self, reserve_a, reserve_b):
        self.initial_a = reserve_a
        self.initial_b = reserve_b
    
    def _execute_batch(self, txs, ordering):
        """Execute transactions in given order, return results."""
        pool = SimpleAMM(self.initial_a, self.initial_b)
        results = []
        
        for idx in ordering:
            tx = txs[idx]
            price_before = pool.price
            if tx['direction'] == 'buy_a':
                received = pool.swap_b_for_a(tx['amount'])
            else:
                received = pool.swap_a_for_b(tx['amount'])
            
            results.append({
                'tx_id': idx, 'trader': tx['trader'],
                'received': received, 'price_before': price_before,
                'price_after': pool.price
            })
        
        return results
    
    def compare_orderings(self, txs):
        """Compare FIFO, attacker-optimal, and random ordering."""
        n = len(txs)
        
        # FIFO: first in, first out
        fifo = self._execute_batch(txs, list(range(n)))
        
        # Attacker-optimal: attacker transactions first and last
        # Assume tx 0 is attacker front-run, tx -1 is attacker back-run
        attacker_order = [0] + list(range(1, n-1)) + [n-1]
        attacker_opt = self._execute_batch(txs, attacker_order)
        
        # Random shuffles (average of 100)
        np.random.seed(42)
        random_results = []
        for _ in range(100):
            order = list(range(n))
            np.random.shuffle(order)
            random_results.append(self._execute_batch(txs, order))
        
        return fifo, attacker_opt, random_results

# Build a realistic block with 8 transactions including an attacker
txs = [
    {'trader': 'ATTACKER (front)', 'direction': 'buy_a', 'amount': 60},
    {'trader': 'Mollen Vek', 'direction': 'buy_a', 'amount': 25},
    {'trader': 'Hask Berrol', 'direction': 'buy_a', 'amount': 80},  # Large victim trade
    {'trader': 'Vagabu Olt', 'direction': 'buy_a', 'amount': 15},
    {'trader': 'Boffa Trent', 'direction': 'buy_a', 'amount': 30},
    {'trader': 'Yasho Krent', 'direction': 'buy_a', 'amount': 10},
    {'trader': 'Serath Kyne', 'direction': 'buy_a', 'amount': 20},
    {'trader': 'ATTACKER (back)', 'direction': 'buy_a', 'amount': 5},
]

sim = FairOrderingSimulator(1000, 1600)
fifo, attacker_opt, randoms = sim.compare_orderings(txs)

print("=== ORDERING COMPARISON ===")
print(f"\nFIFO (fair) — Hask Berrol receives: {fifo[2]['received']:.4f} iron")
print(f"Attacker-optimal — Hask Berrol receives: {attacker_opt[2]['received']:.4f} iron")

# Average random
berrol_random = np.mean([r[2]['received'] for r in randoms])
print(f"Random ordering (avg) — Hask Berrol receives: {berrol_random:.4f} iron")

harm_attacker = fifo[2]['received'] - attacker_opt[2]['received']
harm_random = fifo[2]['received'] - berrol_random
print(f"\nVictim harm from attacker ordering: {harm_attacker:.4f} iron ({harm_attacker/fifo[2]['received']:.1%} worse)")
print(f"Victim harm from random ordering: {harm_random:.4f} iron ({harm_random/fifo[2]['received']:.1%} worse)")
print(f"\nFair ordering (FIFO/encrypted mempool) eliminates ordering-based extraction.")

In [ ]:
# Mitigation strategies comparison
strategies = [
    ('No protection', 'Public mempool, validator discretion', 'None', 100),
    ('Slippage limits', 'User sets max acceptable slippage', 'Partial', 60),
    ('Private mempool', 'Encrypted transactions until inclusion', 'High', 15),
    ('Flashbots/MEV-Share', 'Auction for ordering rights, rebates to users', 'High', 20),
    ('Batch auctions', 'All trades in a batch execute at same price', 'Full', 0),
]

print("MEV Mitigation Strategies:")
print(f"{'Strategy':25s} {'Protection':>11s} {'MEV Remaining':>14s}")
for name, desc, protection, remaining in strategies:
    bar = '█' * (remaining // 5) + '░' * ((100 - remaining) // 5)
    print(f"  {name:23s} {protection:>11s} {remaining:3d}% {bar}")

print(f"\nNo single solution eliminates MEV completely.")
print(f"The ecosystem has converged on MEV-Share (Flashbots): let extractors compete,")
print(f"but rebate a portion of profits back to the users who generated the opportunity.")

## Exercises

### Exercise 1: Optimal Sandwich Size

For a given victim trade size, find the attacker front-run amount that maximizes profit. Test victim trades of 50, 100, and 200 and plot the profit curve.

In [ ]:
victim_sizes = [50, 100, 200]
attacker_range = np.linspace(1, 300, 60)

fig, ax = plt.subplots(figsize=(10, 6))
colors = ['steelblue', 'darkorange', 'firebrick']

for victim_size, color in zip(victim_sizes, colors):
    profits = []
    for atk_size in attacker_range:
        r = simulate_sandwich(1000, 1600, victim_size, atk_size)
        profits.append(r['attacker_profit'])
    
    ax.plot(attacker_range, profits, color=color, linewidth=2,
            label=f'Victim = {victim_size} salt')
    
    # Find optimal
    opt_idx = np.argmax(profits)
    ax.plot(attacker_range[opt_idx], profits[opt_idx], 'o', color=color, markersize=10)
    print(f"Victim={victim_size}: optimal front-run = {attacker_range[opt_idx]:.0f}, "
          f"max profit = {profits[opt_idx]:.3f}")

ax.axhline(y=0, color='black', linestyle='--', alpha=0.3)
ax.set_xlabel('Front-Run Size')
ax.set_ylabel('Attacker Profit')
ax.set_title('Optimal Sandwich Attack Size')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nLarger victim trades create larger extraction opportunities.")
print(f"But each has a sweet spot — too large a front-run eats into its own profits.")

### Exercise 2: MEV Profitability Threshold

Given gas costs, what is the minimum victim trade size that makes sandwich attacks profitable? Sweep victim sizes from 1 to 500 with a fixed gas cost.

In [ ]:
gas_cost = 15  # Fixed gas cost for 2 transactions (front + back)
victim_range = np.linspace(5, 500, 100)
max_profits = []

for v_size in victim_range:
    best = -999
    for atk_size in np.linspace(1, v_size * 2, 30):
        r = simulate_sandwich(1000, 1600, v_size, atk_size)
        net = r['attacker_profit'] - gas_cost
        best = max(best, net)
    max_profits.append(best)

plt.figure(figsize=(10, 5))
plt.plot(victim_range, max_profits, 'b-', linewidth=2)
plt.axhline(y=0, color='firebrick', linestyle='--', linewidth=2)
plt.fill_between(victim_range, max_profits, 0,
                 where=[p > 0 for p in max_profits], color='firebrick', alpha=0.1,
                 label='Profitable MEV zone')
plt.xlabel('Victim Trade Size')
plt.ylabel('Max Net Profit (after gas)')
plt.title(f'MEV Profitability Threshold (gas cost = {gas_cost})')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

for v, p in zip(victim_range, max_profits):
    if p > 0:
        print(f"Sandwich becomes profitable at victim trade ≈ {v:.0f} salt")
        print(f"Smaller trades are not worth attacking (gas > profit).")
        print(f"This is why MEV disproportionately affects large traders.")
        break

### Exercise 3: MEV Concentration

How concentrated is MEV extraction? Compute the Gini coefficient and HHI (Herfindahl-Hirschman Index) for extractor profits.

In [ ]:
# MEV concentration analysis
extractor_profits = mev_log.groupby('extractor')['net_profit'].sum().sort_values()
extractor_profits = extractor_profits[extractor_profits > 0]  # Only profitable extractors

# Gini coefficient
profits = extractor_profits.values
n = len(profits)
if n > 1:
    gini = (2 * np.sum((np.arange(1, n+1) * profits))) / (n * np.sum(profits)) - (n + 1) / n
else:
    gini = 0

# HHI (market share squared, summed)
total = profits.sum()
shares = profits / total
hhi = np.sum(shares ** 2) * 10000  # Normalized to 10000

print(f"MEV Concentration Analysis:")
print(f"  Profitable extractors: {n}")
print(f"  Total net profit: {total:,.1f}")
print(f"  Gini coefficient: {gini:.3f} (0=equal, 1=monopoly)")
print(f"  HHI: {hhi:.0f} (< 1500=competitive, > 2500=concentrated)")

# Lorenz curve
cumulative = np.cumsum(profits) / total
cumulative = np.insert(cumulative, 0, 0)
pop_pct = np.linspace(0, 1, len(cumulative))

plt.figure(figsize=(8, 8))
plt.plot(pop_pct, cumulative, 'b-', linewidth=2, label=f'MEV extraction (Gini={gini:.3f})')
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Perfect equality')
plt.fill_between(pop_pct, cumulative, pop_pct, alpha=0.1, color='blue')
plt.xlabel('Cumulative Share of Extractors')
plt.ylabel('Cumulative Share of Profits')
plt.title('MEV Extraction Concentration')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTop extractor controls {shares[-1]:.0%} of all MEV profit.")
print(f"MEV extraction concentrates toward specialists with better infrastructure.")

## Summary

In this tutorial, we learned:

1. **MEV is the hidden tax of transparent blockchains** — validators who see pending transactions can profit from reordering them
2. **Sandwich attacks** are the most harmful form — front-run + back-run wraps victim trades, extracting value from both sides
3. **Not all MEV is harmful** — arbitrage corrects prices, liquidations maintain protocol health
4. **Detection** relies on pattern matching in transaction logs — looking for the telltale front-run/victim/back-run sequence
5. **Fair ordering** (Flashbots, encrypted mempools, batch auctions) mitigates but doesn't eliminate MEV

**Key insight:** MEV reveals a fundamental tension in transparent systems. The AMM eliminated the need for trusted market-makers — but the block builder became a new kind of market-maker, one who profits from ordering rather than pricing. Auster's response: "Every trust assumption you remove creates a new one. The question is not whether trust exists, but where it lives and what it costs."

---

**Next Tutorial:** Capstone — The Guild Economy. Combine everything: ERC-20 tokens, staking, AMMs, and MEV into a complete economic simulation.

---

> *"Every trust assumption you remove creates a new one. The question is not whether trust exists, but where it lives and what it costs."*
>
> — Brenn Auster